In [1]:
# Data processing libraries
import pandas as pd
import numpy as np

# NLP libraries
import nltk
from nltk.tag import StanfordPOSTagger

# Machine Learning Libraries
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split # Parameter selection
import sklearn_crfsuite
from sklearn_crfsuite import scorers, metrics
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

# Other libraries
import time # Execution time of some blocks
import statistics
from IPython.display import display # For displaying DataFrames correctly in Jupyter
from itertools import chain
import collections
import pickle

# Import our own defined functions
from xlm_parsers_functions import *
from drug_interaction_functions import *
from drug_functions import *
from NER_functions import *
from ortographic_features import *
from context_features import *
from feature_creation_interaction import *
from crf_functions import *

/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/cesc/Anaconda3/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%%time

with open("X_train.txt", "rb") as f:   # Unpickling
    X_train = pickle.load(f)
    
with open("y_train_int.txt", "rb") as f:   # Unpickling
    y_train_int = pickle.load(f)
    
with open("y_train_type.txt", "rb") as f:   # Unpickling
    y_train_type = pickle.load(f)

with open("X_test.txt", "rb") as f:   # Unpickling
    X_test = pickle.load(f)
    
with open("y_test_int.txt", "rb") as f:   # Unpickling
    y_test_int = pickle.load(f)
    
with open("y_test_type.txt", "rb") as f:   # Unpickling
    y_test_type = pickle.load(f)

print('Number of training sentences readed: ', len(X_train))
print('Number of testing sentences readed: ', len(X_test))

Number of training sentences readed:  27792
Number of testing sentences readed:  5716
CPU times: user 1.84 s, sys: 320 ms, total: 2.16 s
Wall time: 2.18 s


In [3]:
%%time

mod1 = trainCRFAndEvaluate(
            X_train = X_train, 
            y_train = y_train_int,
            X_test = X_test,
            y_test = y_test_int,
            labels = ['true', 'false'],
            hyperparam_optim = False)

             precision    recall  f1-score   support

       true      0.507     0.531     0.519       979
      false      0.902     0.893     0.898      4737

avg / total      0.835     0.831     0.833      5716

CPU times: user 28.4 s, sys: 287 ms, total: 28.7 s
Wall time: 28.8 s


In [4]:
y_pred_int = mod1.predict(X_test)

In [5]:
X_train_filtered = []
y_train_filtered = []
for idx,val in enumerate(y_train_int):
    if val==['true']:
        X_train_filtered.append(X_train[idx])
        y_train_filtered.append(y_train_type[idx]) # no we don't want true or false rather than int, mechanism...


X_test_filtered = []
y_test_filtered = []
idx_positive_interaction = []
for idx,val in enumerate(y_pred_int):
    if val==['true']:
        idx_positive_interaction.append(1)
        X_test_filtered.append(X_test[idx])
        y_test_filtered.append(y_test_type[idx]) # no we don't want true or false rather than int, mechanism...
    else:
        idx_positive_interaction.append(0)

print('Number of training sentences: ', len(X_train_filtered))
print('Number of testing sentences: ', len(X_test_filtered))

Number of training sentences:  4021
Number of testing sentences:  1025


In [13]:
%%time

mod2 = trainCRFAndEvaluate(
            X_train = X_train_filtered, 
            y_train = y_train_filtered,
            X_test = X_test_filtered,
            y_test = y_test_filtered,
            labels = ['mechanism', 'int', 'advise', 'effect'],
            hyperparam_optim = False)

             precision    recall  f1-score   support

  mechanism      0.371     0.832     0.513       155
        int      0.464     0.213     0.292        61
     advise      0.403     0.785     0.532       121
     effect      0.387     0.874     0.537       183

avg / total      0.395     0.763     0.500       520

CPU times: user 6.8 s, sys: 74 ms, total: 6.88 s
Wall time: 6.93 s


In [7]:
y_pred_type = mod2.predict(X_test_filtered)

In [16]:
joinResultsFirstSecondModel(y_test_type = y_test_type, y_pred_type = y_pred_type, y_pred_int = y_pred_int)

             precision    recall  f1-score   support

  mechanism      0.371     0.427     0.397       302
     advise      0.403     0.430     0.416       221
     effect      0.387     0.444     0.414       360
        int      0.464     0.135     0.210        96

avg / total      0.393     0.406     0.389       979



In [10]:
import collections

def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(collections.Counter(mod2.state_features_).most_common(15))

print("\nTop negative:")
print_state_features(collections.Counter(mod2.state_features_).most_common()[-15:])



Top positive:
4.077044 advise   first_modal_sentence:should
1.935437 advise   sentence_contains_neg
1.929185 effect   2_grams_bw_entities:- 
1.623370 effect   3_grams_bw_entities:N a
1.334241 int      POS_tags_sentence_simpl:VB-IN-JJ-NN-,-EX-MD-VB-DT-NN-IN-NN-CC-NN-.
1.321691 int      3_grams_bw_entities:int
1.295215 int      POS_tags_sentence_simpl:NN-IN-NN-CC-NN-IN-NN-.
1.267537 advise   POS_tags_sentence_simpl:IN-NN-VB-DT-JJ-NN-IN-NN-,-CC-NN-VB-NN-IN-RB-,-JJ-NN-IN-DT-NN-,-VB-NN-,-CC-NN-VB-JJ-NN-IN-NN-TO-VB-JJ-IN-DT-NN-IN-PR-JJ-NN-VB-.
1.262986 effect   3_grams_bw_entities:pic
1.239781 effect   3_grams_bw_entities:d l
1.217507 effect   POS_tags_sentence_simpl:RB-,-DT-NN-IN-NN-IN-NN-,-JJ-NN-CC-NN-VB-JJ
1.216372 effect   POS_tags_sentence_simpl:NN-:-NN-MD-VB-IN-NN-.
1.179450 advise   ent1_pos_tag_prev_word3:TO
1.150829 effect   POS_tags_sentence_simpl:NN-VB-IN-JJ-NN-VB-DT-NN-IN-NN-,-VB-IN-VB-NN-IN-NN-CC-PR-JJ-NN-WD-MD-VB-NN-.
1.132549 mechanism 3_grams_bw_entities:y f

Top negative:
-1